# Building a panel pipeline

In a few past blog posts, we have given some examples of how to build dashboards using panel.  These have all been one stage examples, but you can actually use panel to build a pipeline of stages with information that is carried over from one stage to the next.  


In this post, I will show you an example of a simple pipeline to give you an idea of how easy it is to put into place.

## Instantiating the pipeline

You will start by importing panel, then instantiating a panel pipeline.  

In [ ]:
import panel as pn

pn.extension()

dag = pn.pipeline.Pipeline(inherit_params=False, debug=True)

## Building Stages

Next, you can add stages to the pipeline.  In order to this, we need build stages to add.  



For a panel pipeline to work, you must create paramterized classes (inherit from the `param.Parameterized` class) and include param objects.  Param is a library that declares types, ranges and documentation for param objects.  In panel, param is used to link a param object with a certain visualization or computation (using the `@param.depends` method); and in a panel pipeline, the param `@param.output` method is used to indicate the parameters to pass from one stage to the next.  The following stage in the pipeline must also have the same parameters declared in order to consume the output from the previous stage.  

In addition, you must also include a `panel` method for each stage that will determine the layout of the widgets you are including in your app.  
  

Below, `stage1` will display a text input widget and a continue button.  The text typed in stage 1 will be passed to the next stage.  In order to do that I have defined a `text` string parameter and an `output` method with the `param.output('text')` decorator.  This indicates that `text` is the output of this stage.

I also want to point out here that I included a `ready` parameter.  These can be useful in order to control when the stage is complete and ready to move to the next.  Later, you will see how it is used in the pipeline.

In [ ]:
import param
class stage1(param.Parameterized):
    
    ready = param.Boolean(
        default=False,
        doc='trigger for moving to the next page',
        )   
    
    text = param.String()
    
    def __init__(self, **params):
        super().__init__(**params)
        self.text_input = pn.widgets.TextInput(name='Text Input', placeholder='Enter a string here...')
        self.continue_button = pn.widgets.Button(name='Continue',button_type='primary')
        self.continue_button.on_click(self.on_click_continue)
        
    def on_click_continue(self, event):
        self.ready=True
    
    @param.output('text')
    def output(self):
        text = self.text_input.value
        return text
        
    def panel(self):
        return pn.Column(pn.WidgetBox(self.text_input,
                  self.continue_button)
                 )
        

`Stage2` is going to display a single line of static text that will display what the user entered in `Stage1`.  Below, you can see that `text` was defined again as a `param.String`.  

In [ ]:
class stage2(param.Parameterized):
    
    
    text = param.String()
    
    def __init__(self, **params):
        super().__init__(**params)
        self.text_display = pn.widgets.StaticText(name='Previously, you typed ', value=self.text, font_size=20)
        
    def panel(self):
        return pn.Column(pn.WidgetBox(self.text_display, height = 50), )

## Adding stages to pipeline

Now that our classes are defined, we can add the stages to the pipeline.  Below, you'll notice that each stage has string input that will serve as the string identifier for this stage, and then the second input is the class being added in this stage.  Here I have added a `"Stage 1"` and `"Stage 2"`.  

Earlier, I mentioned having a `ready` parameter defined.  When adding a stage, you can specify the `ready_parameter` and set `auto_advance` to True, which will cause the next stage to appear when that `ready_parameter` is triggered.  

After adding stages, you will define the relationship between the stages by calling the `define_graph` method.  This will determine the order of stages.  Here we will start with `Stage 1` then move to `Stage 2`.  

In [ ]:
dag.add_stage(
    'Stage 1',
    stage1,
    ready_parameter='ready',
    auto_advance=True
)

dag.add_stage(
            'Stage 2',
            stage2,
            )

dag.define_graph(
            {'Stage 1': 'Stage 2',
             }
            )


example_app = pn.Column(dag.stage).servable()

## View the new pipeline

Let's view our app and confirm it does what we expect:

In [ ]:
example_app

This pipeline was about as simple as you can get.  The point here was to show how easy it is to put these pieces into place.  I have included an example below that includes more complicated stages.  You do everything just the same, but now you will insert the different stages in place of `stage1` and `stage2`.



# Pipeline with more complicated stages

I built a custom NLP pre-processing app with 2 stages that I can also insert into a similar pipeline as above.  I can import these classes from a python file, and insert objects of these classes into my pipeline.  


In [ ]:
# Stage 1

from app.api import PreProcessor


# Stage 2

from app.test_train import trainer


The pipeline is established just as before.  Instead of using `stage1` and `stage2` as my string tags, I will give more meaningful string names for each stage: `Preprocess` and `Testing`.

In [ ]:
import panel as pn
pn.extension()

dag = pn.pipeline.Pipeline(inherit_params=False)

dag.add_stage(
    'Preprocess',
    PreProcessor,
    ready_parameter='ready',
    auto_advance=True
)

dag.add_stage(
            'Testing',
            trainer,
            ready_parameter='ready',
            auto_advance=True,
            )

dag.define_graph(
            {'Preprocess': 'Testing',
             }
            )


SentimentApp = pn.Column(dag.stage).servable()

And now we can view our new app:

In [ ]:
SentimentApp

## Final thoughts

I hope you found this post on panel pipelines enlightening.  You can build much for elaborate stages or pipelines with more than 2 stages. You can even have branching stages that depend on user input (clicking a button to go to one stage or clicking another button to go to a different stage, for example).  Its quite flexible.  I encourage you to try building a panel pipeline app that can be useful in your work or daily life.  Not only is it fun, but I think you'll find panel makes it easy to put these stages into place!